## This asignemnt is scrapting the data from the given wikipedia link
### The output of Dataframe should follow the below:
* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area
* If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

To obtain the data, the link is here:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### The approaching to this assignment
1. Get the data through the link.
2. Create a dataframe and store the extracted data from step1.
3. Process the data, considering ignore ""Borough" 'Not assigned' as removing the entire row. Then group entire data based on the postal code.
4. Show the size after data is processed. 

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as BS
import requests

In [2]:
# using request to get page content
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)

# using beautifulsoup 
soup = BS(page.content, 'html.parser')
#print(soup.prettify)

In [3]:
# extract the table
table = soup.find('table').find('tbody').find_all('tr')
# the first row is the name of table
name = [ i.text.strip() for i in table[0].find_all('th')]
#look at the column name
name

['Postal Code', 'Borough', 'Neighbourhood']

In [4]:
#create a dataframe
postal_code = pd.DataFrame(columns = name)

#extract the rest of value and append into the postal_code
values = []
for row in table[1:]:
    cell = row.find_all('td')    
    #extract the value
    cell = [ i.text.strip() for i in cell]
    postal_code = postal_code.append({name[0]: cell[0], name[1]:cell[1], name[2]:cell[2]}, ignore_index= True)

#have a look on the dataframe
postal_code

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [5]:
#process the data check the number of Borough is Not assigned
postal_code['Borough'].replace('Not assigned', np.nan, inplace = True)
postal_code.isna().sum()

Postal Code       0
Borough          77
Neighbourhood     0
dtype: int64

In [6]:
# we drop the Borough as Nan, SINCE it can be ignored
postal_code.dropna(inplace= True)

#check the datasize and ensure the na is zero
print("After removing \'Not asigned\' in the Borough, the data shape is  :", postal_code.shape)
print(postal_code.isna().sum())

After removing 'Not asigned' in the Borough, the data shape is  : (103, 3)
Postal Code      0
Borough          0
Neighbourhood    0
dtype: int64


In [7]:
# check is there Not assigned in Neighbourhood
print((postal_code['Neighbourhood'] == 'Not assigned').sum())

0


In [8]:
#group the neighbourhood with the same postal code
df = postal_code.groupby(postal_code['Postal Code'].unique() , as_index=False).sum()
df

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [11]:
df.shape

(103, 3)